In [20]:
import requests
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nameparser.parser import HumanName

[nltk_data] Downloading package punkt to /Users/samvance/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/samvance/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/samvance/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/samvance/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#Create dataframe
import pandas as pd
# read saved data = FALSE
df=pd.DataFrame(columns=['Source','Case Number', 'Authors','Discipline', 'Geography', 'Industry', 'Summary'])

In [3]:
# url = 'https://services.hbsp.harvard.edu/api/catalog/products/KEL876-PDF-ENG'
# resp = requests.get(url)
# j = resp.json()
# case_data = j['data']['products'][0]

In [4]:
#PULLING AND ORGANIZING SELECTED CASE INFOMRATION FROM HARVARD'S ONLINE CASE LIBRARIES 

In [5]:
for case_num in range (0 ,1100):
    url = 'https://services.hbsp.harvard.edu/api/catalog/products/KEL'+ str(case_num) + '-PDF-ENG'
    response = requests.get(url)
    try:
        j = response.json()
        case_data = j['data']['products'][0]
        source = case_data['copyRightHolderDisplayName']
        case_number = case_data['coreId']
        author = case_data['authorNames']
        discipline = case_data['majorDiscipline']
        summary = case_data['abstractDescription']
        geography = case_data['geography']
        industry = case_data['industry']
        year = case_data['publicationDate']
        
        
        i = df.shape[0]
        df.loc[i, 'Source'] = source
        df.loc[i, 'Case Number'] = case_number
        df.loc[i, 'Authors'] = author
        df.loc[i, 'Discipline'] = discipline 
        df.loc[i, 'Geography'] = geography
        df.loc[i, 'Industry'] = industry
        df.loc[i, 'Summary'] = summary
        df.loc[i, 'Year'] = year
    except:
        pass

In [13]:
teach_note = df['Summary'].str.contains('Teaching note') | df['Summary'].str.contains('Teaching Note')
df = df[~teach_note]

In [7]:
#ORGANZING DATA FRAME TO FIT NECESSARY API FORMATS: SEPERATING MULTI-AUTHORS AND SPLITTING FIRST AND LAST NAMES 

In [14]:
# new data frame with split value columns 
df['Authors list'] = df["Authors"].str.split(", ")

In [9]:
# authors = pd.DataFrame(df['Author'].str.split(', ').tolist()).stack()
# authors_df = authors.str.split(" ", n = 1, expand = True) 
# authors_df.columns = ['First Name', 'Last Name']
# df.index.name='Case Number'

In [31]:
# Split multiple authors into separate columns
authors_split = pd.DataFrame(df['Authors list'].tolist())
# Stack authors into single column, one row per author (preserving multi-index)
authors_stacked = authors_split.stack()
# Split into first name and last name
authors_stacked_first_last = authors_stacked.str.split(" ", n = 1, expand = True)
authors_stacked_first_last.columns = ["First Name","Last Name"]

In [34]:
from ethnicolr import pred_wiki_name
authors_with_race = pred_wiki_name(authors_stacked_first_last, "Last Name", "First Name")
#Combine race df with complete df as dictionary 

Using TensorFlow backend.
/Users/samvance/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [32]:
def get_gender_api(names_list):
    import json
    from urllib.request import urlopen
    genders = []
    for name in names_list:
        first_name = "First Name"
        myKey = "JsPdwBuUoRBKvUlZMM"
        url = "https://gender-api.com/get?key=" + myKey + "&name=" + first_name
        response = urlopen(url)
        decoded = response.read().decode('utf-8')
        data = json.loads(decoded)
        genders.append(data["gender"])
    return genders

In [ ]:
df.to_csv('Authors-with-demographics.txt', sep='\t') #uses tab separated format

In [ ]:
# Reverse the stacking to combine authors into single row (multi-columns!)
authors_with_race_split = authors_with_race.unstack(1)
# Reverse column splitting to combine into single columns
# First get the column headings (from the multi-index column type)
column_headings = authors_with_race_split.columns.levels[0]
# Then get just the relevant columns
race_columns = column_headings[2:]
for c in race_columns:
    new_column = 'Authors '+c
    temp = df[['Authors list']]
    temp[new_column] = authors_with_race_split[c].values.tolist()
    # Truncate lists to remove NaN values for non-existant co-authors
    df[new_column] = temp.apply(lambda x: x[new_column][:len(x['Authors list'])], axis=1)

In [ ]:
df['Authors list'].to_csv('Authors-with-race.txt', sep='\t') #uses tab separated format

In [ ]:
#PROTAGS 

In [ ]:
import nltk
from nameparser.parser import HumanName

def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)
    person_list = []
    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []

    return (person_list)

In [ ]:
df['Protags']= df['Summary'].apply(get_human_names)

In [ ]:
# new data frame with split value columns 
df['Protags'] = df['Protags'].apply(str).str.split(", ")

In [ ]:
# Split multiple protags into separate columns
protags_split = pd.DataFrame(df['Protags'].tolist())
# Stack protags into single column, one row per author (preserving multi-index)
protags_stacked = protags_split.stack()
# Split into first name and last name
protags_stacked_first_last = protags_stacked.str.split(" ", n = 1, expand = True)
protags_stacked_first_last.columns = ["Protag First","Protag Last"]

In [ ]:
protags_with_race = pred_wiki_name(protags_stacked_first_last, "Last Name", "First Name")

In [ ]:
# Reverse the stacking to combine authors into single row (multi-columns!)
protags_with_race_split = authors_with_race.unstack(1)
# Reverse column splitting to combine into single columns
# First get the column headings (from the multi-index column type)
column_headings = protags_with_race_split.columns.levels[0]
# Then get just the relevant columns
race_columns = column_headings[2:]
for c in race_columns:
    new_column = 'Protags '+c
    temp = df[['Protags list']]
    temp[new_column] = protags_with_race_split[c].values.tolist()
    # Truncate lists to remove NaN values for non-existant co-authors
    df[new_column] = temp.apply(lambda x: x[new_column][:len(x['Protags list'])], axis=1)